In [11]:
import shapely.geometry as geom
import geopandas as gpd
from generate_od.worldpop import worldpop

# out_path = "assets/fukuoka_prefecture.shp"
out_path = "assets/fukuoka_city.shp"
# download example shapefile for Fukuoka Prefecture, Japan
# GADM v4.1 shapefiles for Japan
url = "https://geodata.ucdavis.edu/gadm/gadm4.1/shp/gadm41_JPN_shp.zip"

In [6]:
fukuoka_gpd = gpd.read_file(url, layer="gadm41_JPN_1")
fukuoka_pref = fukuoka_gpd[fukuoka_gpd["NAME_1"] == "Fukuoka"].to_crs("EPSG:4326")

In [9]:
# Level 2 = municipalities (cities, towns, etc.)
gdf_lvl2 = gpd.read_file(url, layer="gadm41_JPN_2")

# Optional: inspect what NAME_2 looks like for Fukuoka prefecture
# print(gdf_lvl2[gdf_lvl2["NAME_1"] == "Fukuoka"]["NAME_2"].unique())
fukuoka_city = gdf_lvl2[
    (gdf_lvl2["NAME_1"] == "Fukuoka") &
    (gdf_lvl2["NAME_2"] == "Fukuoka")
].copy()

# Make sure CRS is EPSG:4326 (what worldpop() expects)
fukuoka_city = fukuoka_city.to_crs("EPSG:4326")

In [12]:
import os

# make sure assets folder exists
os.makedirs("assets", exist_ok=True)
# save shapefile locally
# fukuoka_pref.to_file(out_path)  # default driver is ESRI Shapefile
fukuoka_city.to_file(out_path)  # default driver is ESRI Shapefile

print("saved shapefile to", out_path)


saved shapefile to assets/fukuoka_city.shp


In [15]:
# Load your regional boundaries (shapefile or GeoDataFrame)
area_shp = gpd.read_file(out_path)

# Ensure CRS is set (will be converted to WGS84 internally)
if area_shp.crs is None:
    area_shp = area_shp.set_crs("EPSG:4326")

print("CRS:", area_shp.crs)
print("Bounds:", area_shp.total_bounds)
print(area_shp.geometry.iloc[0].geom_type)
print(area_shp.is_valid)

# Fetch population data for all regions
population_features = worldpop(area_shp, num_proc=1)

# The result contains [population, area_size] for each region
print(f"Population data shape: {population_features.shape}")
print(f"Features: [population_count, area_km2]: {population_features}")

CRS: EPSG:4326
Bounds: [130.032227    33.42413309 130.49566699  33.874367  ]
MultiPolygon
0    True
dtype: bool


 -- Population of regions:   0%|          | 0/1 [00:00<?, ?it/s]

https://worldpop.arcgis.com/arcgis/rest/services/WorldPop_Total_Population_100m/ImageServer/exportImage?f=image&format=tiff&noData=0&
https://worldpop.arcgis.com/arcgis/rest/services/WorldPop_Total_Population_100m/ImageServer/exportImage?f=image&format=tiff&noData=0&bbox=14475121.296355376,3951736.8055680776,14526711.200389368,4011945.0329066785&bboxSR=3857&imageSR=3857&pixelSize=100


 -- Population of regions: 100%|██████████| 1/1 [00:01<00:00,  1.29s/it]

Population data shape: (1, 2)
Features: [population_count, area_km2]: [[  0.         332.01259489]]


In [3]:
# Tiny box over central London
poly = geom.box(-0.2, 51.45, -0.1, 51.55)
gdf = gpd.GeoDataFrame({"id": [1]}, geometry=[poly], crs="EPSG:4326")

population_features = worldpop(gdf, num_proc=1)
print(population_features)

 -- Population of regions:   0%|          | 0/1 [00:00<?, ?it/s]

https://worldpop.arcgis.com/arcgis/rest/services/WorldPop_Total_Population_100m/ImageServer/exportImage?f=image&format=tiff&noData=0&
https://worldpop.arcgis.com/arcgis/rest/services/WorldPop_Total_Population_100m/ImageServer/exportImage?f=image&format=tiff&noData=0&bbox=-22263.898158654716,6701282.868773621,-11131.949079327358,6719165.106882159&bboxSR=3857&imageSR=3857&pixelSize=100


 -- Population of regions: 100%|██████████| 1/1 [00:01<00:00,  1.43s/it]

[[ 0.         77.27040313]]
